In [1]:
import numpy as np
import pandas as pd
import json
import os
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
import time
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import pickle

binary=False

root='data'

labelFile=pd.read_csv('c:/data/label.csv')
nameList=labelFile['ID']




In [2]:
personList=os.listdir(root)
names=pd.DataFrame()
data=pd.DataFrame()
for personDir in personList:
    if personDir.__contains__('.'):
        continue
    person='data/'+personDir
    #person='data/1B004_10013WLEM(1)'

    jsonFiles = os.listdir(person)
    if jsonFiles.__len__()==0:
        continue
    name=person.split('/')[1].split('_')[0]
    if nameList.str.contains(name).sum()==0:
        continue
    personData=pd.DataFrame()
    for jsonFile in jsonFiles:
        if not jsonFile.__contains__('.json'):
            continue

        with open((person+'/'+jsonFile), "r", encoding='UTF-8') as st_json:
            st_python = json.load(st_json)
            st_json.close()

        row=st_python['annotations'][0]['keypoints']
        row=pd.Series(row)
        personData=pd.concat([personData, row], axis=1)
    means=personData.T.mean()
    stds=personData.T.std()
    name=pd.Series(name)
    print(name)
    data=pd.concat([data,stds],axis=1)
    names=pd.concat([names,name],axis=1)

data.columns=names.T[0].to_list()



0    1B004
dtype: object
0    1B004
dtype: object
0    1B005
dtype: object
0    1B005
dtype: object
0    1B006
dtype: object
0    1B006
dtype: object
0    1B008
dtype: object
0    1B008
dtype: object
0    1B008
dtype: object
0    1B011
dtype: object
0    1B011
dtype: object
0    1B011
dtype: object
0    1B011
dtype: object
0    1B011
dtype: object
0    1B011
dtype: object
0    1B013
dtype: object
0    1B013
dtype: object
0    1B013
dtype: object
0    1B013
dtype: object
0    1B013
dtype: object
0    1B013
dtype: object
0    1B013
dtype: object
0    1B013
dtype: object
0    1B013
dtype: object
0    1B014
dtype: object
0    1B014
dtype: object
0    1B014
dtype: object
0    1B014
dtype: object
0    1B014
dtype: object
0    1B015
dtype: object
0    1B015
dtype: object
0    1B015
dtype: object
0    1B015
dtype: object
0    1B015
dtype: object
0    1B015
dtype: object
0    1B015
dtype: object
0    1B015
dtype: object
0    1B015
dtype: object
0    1B016
dtype: object
0    1B016
dtype: object


In [3]:

labels=pd.DataFrame()
for name in data.columns:
    label=labelFile[labelFile['ID']==name]['SEVERITY']
    if binary:
        if label.size==0 or label.values[0]!=0:
            label=pd.Series(1)
        else:
            label=pd.Series(0)
    labels=pd.concat([labels,label])

    #print(name," : ",label.values[0])




x=data.T
y=labels[0]



In [4]:

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=10000)
# 모델링 설정 
clfs=[
    SVC(kernel="linear", C=0.025,random_state=1),
    DecisionTreeClassifier(max_depth=5,random_state=1),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1,random_state=1),
    MLPClassifier(alpha=1, max_iter=5000,random_state=1),
    AdaBoostClassifier()
]

max_score=-1


In [5]:
#saving model to disk # my test
#내가 원하는 모형으로 저장하기 여기서는 svc 모형을 적용하였음.
pickle.dump(SVC, open('model1.pkl', 'wb'))


#내가 원하는 모형으로 저장하기 여기서는 랜덤포레스트 모형을 적용하였음.
pickle.dump(RandomForestClassifier, open('model2.pkl', 'wb'))


#내가 원하는 모형으로 저장하기 여기서는 랜덤포레스트 모형을 적용하였음.
pickle.dump(DecisionTreeClassifier, open('model3.pkl', 'wb'))




In [6]:

for clf in clfs:
    start_ts=time.time()

    print('------------------------')
    print(clf)
    score=cross_val_score(clf, x_train, y_train, scoring="accuracy", cv=5).mean()
    print(score)
    y_pred = cross_val_predict(clf, x_train, y_train, cv=5)
    conf_mat = confusion_matrix(y_train, y_pred)
    print(conf_mat)
    runtime=time.time()-start_ts
    print('runtime: ',runtime)

    if score>max_score:
        max_score=score
        selectedClf=clf

f2 = open("result_train.txt", 'w')


print('selected algorithm: ',selectedClf)
f2.write('selected algorithm: '+str(selectedClf)+'\n')
selectedClf.fit(x_train,y_train)
y_pred=selectedClf.predict(x_test)
print('accuracy: ', sum(y_pred==y_test)/len(y_pred))
f2.write('accuracy: '+ str(sum(y_pred==y_test)/len(y_pred))+'\n')
conf_mat=confusion_matrix(y_test,y_pred)
print(conf_mat)
f2.write(str(conf_mat))
f2.close()
with open('model.pkl','wb') as f:
    pickle.dump(selectedClf,f)
    f.close()

------------------------
SVC(C=0.025, kernel='linear', random_state=1)
0.7647597977243994
[[275  26   2   1]
 [ 29 130  13   5]
 [  4  37  11   1]
 [  1  13   0  13]]
runtime:  0.5018715858459473
------------------------
DecisionTreeClassifier(max_depth=5, random_state=1)
0.8057522123893804
[[275  26   0   3]
 [  8 165   4   0]
 [  2  38  11   2]
 [  3  20   3   1]]
runtime:  0.04482412338256836
------------------------
RandomForestClassifier(max_depth=5, max_features=1, n_estimators=10,
                       random_state=1)
0.805736409608091
[[279  25   0   0]
 [  8 165   3   1]
 [  2  43   8   0]
 [  0  26   1   0]]
runtime:  0.12690424919128418
------------------------
MLPClassifier(alpha=1, max_iter=5000, random_state=1)
0.8324273072060683
[[288  14   2   0]
 [ 10 143  17   7]
 [  1  24  25   3]
 [  1  12   3  11]]
runtime:  3.624776601791382
------------------------
AdaBoostClassifier()
0.5548040455120101
[[185  37   0  82]
 [ 31 122   5  19]
 [  3  37   4   9]
 [  3  19   0   5]

In [7]:
# MLP모형의 정확도는 85% 

In [8]:
##위 모델 학습

In [9]:
x_train.head()

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
A243,9.906359,28.968829,0.0,8.415293,23.749666,0.0,13.828384,23.429951,0.0,11.668354,...,0.0,36.390705,98.134974,0.0,39.130860,89.686213,0.0,42.100026,89.914745,0.0
A060,5.099152,12.960947,0.0,10.184980,7.680705,0.0,6.270585,8.139591,0.0,19.285260,...,0.0,26.497260,97.203078,0.0,45.697895,86.549157,0.0,43.826272,86.316187,0.0
A117,78.864680,8.257891,0.0,77.716407,10.213717,0.0,80.945643,9.349579,0.0,72.692532,...,0.0,69.616571,47.586724,0.0,56.313525,68.659340,0.0,63.845326,66.842649,0.0
A264,11.517879,24.517457,0.0,7.408230,18.858675,0.0,16.773185,19.468266,0.0,8.218959,...,0.0,45.495165,99.671574,0.0,37.590740,88.927879,0.0,46.208155,89.702185,0.0
1E019,46.695964,15.064064,0.0,43.560472,14.223463,0.0,48.452253,14.444868,0.0,39.515974,...,0.0,55.416414,68.637481,0.0,48.491066,81.481368,0.0,52.193955,79.868204,0.0


In [10]:
conf_mat

array([[141,   4,   0,   0],
       [  6,  50,   3,   5],
       [  2,  11,  11,   2],
       [  1,   2,   0,   3]], dtype=int64)

In [11]:
##아래 모델 예측

In [12]:
import pandas as pd
import pickle
import json
import os

file3=None
f = open("c:/data/init_predict.txt", 'r')
file = f.readline()
print(file)
f.close()
root2='c:/testt'

#root2 에 test / testt 두가지 데이터 저장공간을 통해 구분하여 테스트

test


In [13]:
# 모델을 적합 
# 여기서 model.pkl에 대한 생성방법에 대해서는 나와있지 않기에 찾아야함

f2 = open("c:/data/result_predict.txt", 'w')

with open('model.pkl','rb') as f:
    model=pickle.load(f)

#jsonFiles = os.listdir(file)
## 안되길래 아래거로바꿔봄
##

In [14]:
#########################################test 다른모형을 사용해봄############################

#f2 = open("c:/data/result_predict.txt", 'w')

#with open('model3.pkl','rb') as f:
#   model=pickle.load(f)

## predict_proba() missing 1 required positional argument: 'X' 라는 에러문구가 뜸 랜덤포레스트 svc 등  적합한 모형이 아니기 때문에 


In [15]:
# 즉 mlpclassifier 모델을 활용하였다.

model3 = pickle.load(open('model.pkl', 'rb'))
model3

MLPClassifier(alpha=1, max_iter=5000, random_state=1)

In [16]:
######## 새로 삽입
###person 도 됬음
###root2 가 정상경로여야함. 근데 error   찐경로는 file
jsonFiles = os.listdir(root2)

In [17]:
#jsonFiles = os.listdir(file)

In [18]:
## 132개의 json 형식의 test data set을 불러오기
personData=pd.DataFrame()
for jsonFile in jsonFiles:
    if not jsonFile.__contains__('.json'):
        continue
### 'c:/'+test >>> root2
    with open((root2+'/'+jsonFile), "r", encoding='UTF-8') as st_json:
        st_python = json.load(st_json)
        st_json.close()

    row=st_python['annotations'][0]['keypoints']
    row=pd.Series(row)
    personData=pd.concat([personData, row], axis=1)

personData

,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,345,346,346,346,345,346,346,346,346,346,...,645,645,646,645,646,645,645,645,644,644
1,68,68,68,67,68,67,68,68,68,68,...,164,164,164,164,164,165,166,166,165,165
2,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
3,349,350,349,349,349,349,350,350,350,350,...,651,651,651,651,651,651,650,651,650,651
4,64,63,64,62,64,63,64,64,64,63,...,158,158,158,158,157,159,159,159,159,158
5,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
6,342,342,341,342,341,341,342,342,341,343,...,640,639,640,640,639,639,639,638,639,638
7,65,65,64,64,65,64,64,64,64,64,...,158,158,158,158,157,159,159,159,159,158
8,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
9,357,357,357,356,356,357,357,357,358,358,...,661,660,660,659,660,660,660,660,660,660


In [19]:
# x는 전치행렬
# 132개의 컬럼을 온전히 다 읽어내기 위해 전치행렬을 사용
x=pd.DataFrame(personData)
x=x.T

x

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,345,68,2,349,64,2,342,65,2,357,...,2,346,188,2,378,230,2,348,233,2
0,346,68,2,350,63,2,342,65,2,357,...,2,346,188,2,379,230,2,348,232,2
0,346,68,2,349,64,2,341,64,2,357,...,2,346,188,2,378,229,2,348,232,2
0,346,67,2,349,62,2,342,64,2,356,...,2,345,187,2,379,231,2,348,232,2
0,345,68,2,349,64,2,341,65,2,356,...,2,345,189,2,378,230,2,347,232,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,645,165,2,651,159,2,639,159,2,660,...,2,627,346,2,659,408,2,627,408,2
0,645,166,2,650,159,2,639,159,2,660,...,2,627,348,2,659,409,2,627,410,2
0,645,166,2,651,159,2,638,159,2,660,...,2,627,348,2,659,410,2,627,410,2
0,644,165,2,650,159,2,639,159,2,660,...,2,627,348,2,659,410,2,627,411,2


In [20]:
result=model.predict_proba(x)
print(result)

[[9.99924131e-01 7.50208955e-05 8.48214093e-07 5.41327674e-52]
 [9.99895289e-01 1.03578274e-04 1.13297584e-06 1.02515699e-51]
 [9.99898805e-01 9.98505290e-05 1.34442232e-06 7.37993236e-52]
 ...
 [9.74998581e-01 5.50581572e-08 2.50013639e-02 2.68682361e-91]
 [9.74899031e-01 8.39667088e-08 2.51008848e-02 2.29379572e-91]
 [9.70291154e-01 1.26289175e-07 2.97087202e-02 1.87363291e-91]]


In [21]:
#전치행렬전
#result2=model.predict_proba(personData)
#print(result2)


In [22]:
#뇌졸증 편마비 환자 진단 및 중증도분류 모형 


#Normal (일반)
#Mild(경증)
#Moderate(중등도)
#Severe(중증)
for probs in result[0]:
    print('%0.4f'%probs)
    f2.write('%0.4f'%probs+'\n')
f2.close()



0.9999
0.0001
0.0000
0.0000


In [23]:
#이미지 데이터에 대한 확보가 가능한지 문의
#부재중 220308 오전 오후 all
#이메일 회답받음 오프라인 안심존에서만 데이터 다운가능
